In [ ]:
# Installer tensorflow
pip install tensorflow

In [ ]:
# importer les packages
import os
import numpy as np
import tensorflow as tf
from glob import glob

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten
from keras.models import Model

In [ ]:
# importer la base de données
data_path='/content/drive/My Drive/Kaggle/pneumonia_data/chest_xray'

In [ ]:
#spécifier le chemin des bases de données train et test
train_dir=os.path.join(data_path,'train')
test_dir=os.path.join(data_path,'test')

In [ ]:
# Préparer le modèle de base
IMG_SHAPE=(224,224,3)
base_model=VGG16(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')
base_model.summary()

58892288/58889256 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [ ]:
# bloquer le modèle de base
base_model.trainable=False

In [ ]:
base_model.output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 7, 7, 512) dtype=float32>

In [ ]:
#Ajouter les couches de sorties
x=Flatten()(base_model.output)
prediction=Dense(2,activation='softmax')(x)

In [ ]:
#création du modèle global
model=Model(inputs=base_model.input,outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# Creer les photo augmentées
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

trainning_set=train_datagen.flow_from_directory(train_dir,target_size=(224,224),batch_size=32,class_mode='categorical')
test_set=test_datagen.flow_from_directory(test_dir,target_size=(224,224),batch_size=32,class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# Appliquer le modèle
model.fit_generator(trainning_set,epochs=5,validation_data=test_set)

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 3 could not be retrieved. It could be because a worker has died.
  UserWarning)


  4/163 [..............................] - ETA: 2:57:12 - loss: 1.5996 - accuracy: 0.6328